In [1]:
import numpy as np
import cv2

In [2]:
img = cv2.imread('scene4.jpg')

img_height = img.shape[0]
img_width = img.shape[1]

img_resize = cv2.resize(img,(300,300))     # caffe model input shape is 300x300
img_blob = cv2.dnn.blobFromImage(img_resize,0.007843,(300,300),127.5)
#recommended scale factor is 0.007843, width,height of blob is 300,300, mean of 255 is 127.5 

In [3]:
class_labels = ["background", "aeroplane", "bicycle", "bird", "boat","bottle", "bus", "car", "cat", "chair",
                "cow", "diningtable","dog", "horse", "motorbike", "person", "pottedplant", "sheep","sofa",
                "train", "tvmonitor"]  

In [4]:
mobilenetssd = cv2.dnn.readNetFromCaffe('mobilenetssd.prototext','mobilenetssd.caffemodel')
mobilenetssd.setInput(img_blob)
obj_detections = mobilenetssd.forward()   # detection predictions by the model using forward()

In [5]:
no_of_detections = obj_detections.shape[2]
no_of_detections

100

In [6]:
for i in np.arange(0, no_of_detections):
    prediction_confidence = obj_detections[0, 0, i, 2]
    
    if prediction_confidence >= 0.30:
    
        predicted_class_index = int(obj_detections[0, 0, i, 1])
        predicted_class_label = class_labels[predicted_class_index]
        
        #obtain the bounding box co-oridnates for actual image from resized image size
        bounding_box = obj_detections[0, 0, i, 3:7] * np.array([img_width, img_height, img_width, img_height])
        (start_x_pt, start_y_pt, end_x_pt, end_y_pt) = bounding_box.astype("int")
        
        
        predicted_class_label = "{}: {:.2f}%".format(class_labels[predicted_class_index], prediction_confidence * 100)
        cv2.rectangle(img, (start_x_pt, start_y_pt), (end_x_pt, end_y_pt), (0,255,0), 2)
        cv2.putText(img, predicted_class_label, (start_x_pt, start_y_pt-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)


cv2.imshow("xyz", img)
cv2.waitKey(0)                 
cv2.destroyAllWindows()

---------------